In [2]:
import tensorflow as tf

node1 = tf.constant(10, dtype=tf.float32)
node2 = tf.constant(20, dtype=tf.float32)

node3 = node1 + node2

# 그래프를 실행시키기 위해서는 session필요
# 2.x 버전에선 session 없음

sess = tf.Session()

print(sess.run(node3))

30.0


In [5]:
# Data Flow Graph에 입력값을 주는 방법
# placeholder 사용

node1 = tf.placeholder(dtype=tf.float32)
node2 = tf.placeholder(dtype=tf.float32)

node3 = node1 + node2

sess = tf.Session()

# placeholder에 데이터를 입력하기 위해서 feed_dict를 이용
reuslt = sess.run(node3, feed_dict={node1 : 10, node2 : 20})
print(reuslt)

30.0


In [16]:
# Tensorflow를 이용해서 Simple Linear Regression
import numpy as np
import pandas as pd
import tensorflow as tf

# 1.traing data set
x_data = np.array([1,2,3,4,5]).reshape(-1,1)
t_data = np.array([3,5,7,9,11]).reshape(-1,1)

# 2. placeholder
# 2차원이상이면 반드시 형태 지정해야함
# 이때 x를 이렇게 지정을 하면 항상 입력데이터가 (5,1)이어야 한다
# 그래서 None으로 지정 어떤 값이 들어와도 상관하지 않는다는 의미
X = tf.placeholder(shape=[None,1], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

# 3. Weight & bias  (y = Wx + b)
W = tf.Variable(tf.random.normal([1, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# 4. Hypothesis or predict model
H = tf.matmul(X, W) + b # y = Wx + b 2차원으로 처리 > y = X dot W + b

# 5. W, b를 구하기 위해 평균제곱오차를 이용한 최소제곱법을 통해 loss func 정의
loss = tf.reduce_mean(tf.square(H - T))

# 6. train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)

# 7. session 만들기 & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 2.x 넘어오면서 삭제

# 8. 학습을 진행
# 반복학습을 진행(1 epoch : training data set 전체를 이용해 1번 학습하는 것)
for step in range(30000):
    
    # train은 W, b를 최적화 하는 노드 그래서 결과값의 의미가 없다 
    # 그래서 _로 지정
    _,W_val,b_val,loss_val = sess.run([train,W,b,loss], feed_dict={X : x_data, 
                                                                   T : t_data})
    if step % 3000 == 0:
        print('W : {}, b : {}, loss : {}'.format(W_val, b_val, loss_val))
        
# 9. 학습이 종료된 후 최적의 W와 b가 계산되고
# 이를 이용한 model이 완성

result = sess.run(H, feed_dict={X : [[9]]})
print('예측값은 : {}'.format(result))

W : [[-1.6204088]], b : [-0.26986837], loss : 173.378173828125
W : [[2.0249164]], b : [0.91004866], loss : 0.001472763018682599
W : [[2.0090432]], b : [0.96736133], loss : 0.00019390511442907155
W : [[2.003282]], b : [0.9881446], loss : 2.5582785383448936e-05
W : [[2.0011926]], b : [0.9956871], loss : 3.3851733860501554e-06
W : [[2.0004325]], b : [0.9984341], loss : 4.462816889372334e-07
W : [[2.000166]], b : [0.99941134], loss : 6.332021484922734e-08
W : [[2.0000782]], b : [0.999733], loss : 1.3282260624691844e-08
W : [[2.000052]], b : [0.9998292], loss : 5.604397390612803e-09
W : [[2.000052]], b : [0.9998292], loss : 5.604397390612803e-09
예측값은 : [[19.000296]]


- Simple Linear Regression with python

In [27]:
import numpy as np

# 1. training data set
x_data = np.array([1,2,3,4,5]).reshape(-1,1)
t_data = np.array([3,5,7,9,11]).reshape(-1,1)

# 2. Weight & bias
W = np.random.rand(1,1)
b = np.random.rand(1)

# 3. Hypothesis or predict model
def predict(x):
    
    y = np.dot(x,W) + b
    
    return y

# 4. loss function
def loss_func(input_obj):
    # input_obj = [W, b]
    
    input_W = input_obj[0]
    input_b = input_obj[1]
    
    y = np.dot(x_data, input_W) + input_b
    
    return np.mean(np.power((t_data - y), 2))

# 5. 편미분 함수
def numerical_derivative(f, x):
    
    delta_x = 1e-4
    derivative_x = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished:
        
        idx = it.multi_index 
        
        tmp = x[idx] 
        x[idx] = tmp + delta_x
        fx_plus_delta = f(x) 
        
        x[idx] = tmp - delta_x
        fx_minus_delta = f(x) 
        
        derivative_x[idx] = (fx_plus_delta - fx_minus_delta) / (2 * delta_x)
        
        x[idx] = tmp
        
        it.iternext()
        
    return derivative_x

# 6. learning rate and learning
learning_rate = 1e-4

for step in range(300000):
    # reval() - 1차원으로 변경
    input_param = np.concatenate((W.ravel(), b.ravel()), axis=0) # [W b]
    derivative_result = learning_rate * numerical_derivative(loss_func, input_param)
    
    # 연산을 위해 slicing을 해서 ndarray 형태 유지
    W = W - derivative_result[:1].reshape(1,1)
    b = b - derivative_result[1:]
    
    if step % 30000 == 0:
        print('W : {}, b : {}'.format(W,b))

# 7. predict
print(predict(np.array([9]).reshape(-1,1)))

W : [[0.89169665]], b : [0.66069375]
W : [[2.00301557]], b : [0.98911284]
W : [[2.00109361]], b : [0.99605172]
W : [[2.0003966]], b : [0.99856814]
W : [[2.00014383]], b : [0.99948073]
W : [[2.00005216]], b : [0.99981168]
W : [[2.00001892]], b : [0.99993171]
W : [[2.00000686]], b : [0.99997523]
W : [[2.00000249]], b : [0.99999102]
W : [[2.0000009]], b : [0.99999674]
[[19.00000176]]


- Simple Linear Regression with sklearn

In [26]:
import numpy as np
from sklearn import linear_model

# 1. training data set
x_data = np.array([1,2,3,4,5]).reshape(-1,1)
t_data = np.array([3,5,7,9,11]).reshape(-1,1)

# 2. linear regression model 생성
model = linear_model.LinearRegression()

# 3. learning
model.fit(x_data, t_data)

# 4. Weight, bias 확인
print('W : {}, b : {}'.format(model.coef_, model.intercept_))

# 5. predict
print(model.predict([[9]]))

W : [[2.]], b : [1.]
[[19.]]
